In [1]:
from google.colab import drive
import os
import gc
import shutil
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
import math

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

In [2]:
drive.mount('/content/drive')

In [ ]:
dataset_dir = '/content/drive/MyDrive/mask'
batch_size = 32
init_learning_rate = 0.0001
epochs = 10
model_name = 'mask_detector_large.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # CUDA가 사용 가능한 경우 GPU 사용

In [ ]:
train_transform = transforms.Compose([
  transforms.RandomRotation(degrees = 15),
  transforms.RandomHorizontalFlip(),
  # transforms.RandomAffine(degrees = 0, shear = 0.2),
  transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
  # transforms.RandomPerspective(distortion_scale=0.2),
  # transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),

  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root = dataset_dir, transform=train_transform)

In [ ]:
dataset

In [ ]:
classes = dataset.classes # ['combined_with_mask', 'combined_without_mask']
classes

In [ ]:
image_counting = {}
for label in classes:
  class_path = os.path.join(dataset_dir, label)
  image_counting[label] = len(os.listdir(class_path))

In [ ]:
image_counting

In [ ]:
train_size = int(0.8 * (len(dataset)))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

test_dataset.dataset.transform = test_transform

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers=4)

model = models.mobilenet_v2(pretrained = True)
num_classes = 2
# model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.classifier[1] = nn.Sequential(
                      nn.Linear(1280, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 32),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(32, num_classes),
                      nn.Softmax(dim=1))

model = model.to(device)

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = init_learning_rate)

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
  model.train()
  train_loss = 0.0
  correct = 0
  total = 0
  train_bar = tqdm(train_loader, desc = 'Training', total = len(train_loader), leave=False)

  for inputs, labels in train_bar:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    predicted = torch.argmax(outputs, axis = 1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

    train_bar.set_postfix(Train_loss = train_loss/len(train_loader), Train_acc = correct/total)

  train_losses.append(train_loss / len(train_loader))
  train_accuracies.append(correct / total)

  torch.cuda.empty_cache()
  gc.collect()

  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0
  test_bar = tqdm(test_loader, desc = 'Validing', total = len(test_loader), leave=False)

  with torch.no_grad():
    for inputs, labels in test_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        predicted = torch.argmax(outputs, axis = 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        test_bar.set_postfix(val_loss = val_loss/len(test_loader), val_acc = correct/total)

  val_losses.append(val_loss / len(test_loader))
  val_accuracies.append(correct / total)

  print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}')

torch.save(model.state_dict(), model_name)

In [ ]:
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, epochs), train_losses, label = 'train loss')
plt.plot(np.arange(0, epochs), val_losses, label = 'train loss')
plt.plot(np.arange(0, epochs), train_accuracies, label = 'train acc')
plt.plot(np.arange(0, epochs), val_accuracies, label = 'val acc')
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

plt.savefig('result_plot.jpg')
plt.show()

In [ ]:
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
  for inputs, labels in test_loader:
    inputs = inputs.to(device)
    outputs = model(inputs)
    preds = torch.argmax(outputs, axis = 1)
    all_labels.extend(labels.cpu().numpy())
    all_preds.extend(preds.cpu().numpy())

In [ ]:
print('Shape of all_labels:', np.array(all_labels).shape)
print('Shape of all_preds:', np.array(all_preds).shape)

In [ ]:
print(classification_report(np.array(all_labels), np.array(all_preds)))